## 0604
- kaggle에서 3진분류를 한 결과, __RF, GradientBoostingClassifier__ 이 두개가 가장 결과가 좋았다.  
- 3~9분류, 이진분류에서도 실험해보고자 함.  
- 이전에 svm을 실험했을 때 11개 특성을 모두 이용하는게 가장 잘 나왔었음. 그리고 차원축소 안하는게 가장 결과가 좋았었었다. 따라서 이번에는 test할겸 11개를 다 넣어서 실험함. 따로 처리(커널)등을 하지 않고 그냥 분류 결과만 비교함

In [1]:
!pip install xgboost

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support


In [2]:
!pip install --upgrade pip

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages (20.1.1)


In [3]:
# Data manipulation
import numpy as np
import pandas as pd

# Data pre-processing
from sklearn.preprocessing import StandardScaler as ss

# Dimensionality reduction
from sklearn.decomposition import PCA

#  Data splitting and model parameter search
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import plot_importance


In [4]:
data=pd.read_csv("./data/binary_winequalityN.csv")
#dataw=pd.read_csv("./data/1_white.csv")
#datar=pd.read_csv("./data/1_red.csv")

data=data.dropna(axis=0)  ##없애버림, 평균값 넣으려면 axis=0대신에 inplate=True 사용
#dataw=dataw.dropna(axis=0)

print "wine 데이터셋의 크기:" ,data.shape
print "\nwine 데이터셋의 키:", data.keys()

wine 데이터셋의 크기: (6463, 14)

wine 데이터셋의 키: Index([u'type', u'fixed acidity', u'volatile acidity', u'citric acid',
       u'residual sugar', u'chlorides', u'free sulfur dioxide',
       u'total sulfur dioxide', u'density', u'pH', u'sulphates', u'alcohol',
       u'quality', u'binaryQuality'],
      dtype='object')


데이터셋의 키
['type', 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6463 entries, 0 to 6496
Data columns (total 14 columns):
type                    6463 non-null object
fixed acidity           6463 non-null float64
volatile acidity        6463 non-null float64
citric acid             6463 non-null float64
residual sugar          6463 non-null float64
chlorides               6463 non-null float64
free sulfur dioxide     6463 non-null float64
total sulfur dioxide    6463 non-null float64
density                 6463 non-null float64
pH                      6463 non-null float64
sulphates               6463 non-null float64
alcohol                 6463 non-null float64
quality                 6463 non-null int64
binaryQuality           6463 non-null int64
dtypes: float64(11), int64(2), object(1)
memory usage: 757.4+ KB


# 훈련 데이터와 테스트 데이터

In [6]:
#x_train에 적재하기 위한 2차원 배열의 특성 dataset을 만든다.
#6463*11(?) 사이즈

#실험의 편의성을 위해 data마다 이름을 붙였다.
f1=data['fixed acidity']
f2=data['volatile acidity']
f3=data['citric acid']
f4=data['residual sugar']
f5=data['chlorides']
f6=data['free sulfur dioxide']
f7=data['total sulfur dioxide']
f8=data['density']
f9=data['pH']
f10=data['sulphates']
f11=data['alcohol']


c_data=np.c_[f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11]

---

#### 원본 

In [7]:
## 원본

X_train, X_test, y_train, y_test = train_test_split(c_data, data['quality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''
X_train, X_test, y_train, y_test = train_test_split(c_data, data['binaryQuality'],test_size=0.3,random_state=0)
X_test, X_val, y_test, y_val  = train_test_split(X_test, y_test,test_size=0.5,random_state=0)
'''
print "X_train 크기: ", X_train.shape #(#(numbe) of data*75%, # of features)
print "y_train 크기: ", y_train.shape #x트레인 데이터에 대한 정답
print "X_val 크기: ", X_val.shape
print "y_val 크기: ", y_val.shape
print "X_test 크기: ", X_test.shape
print "y_test 크기: ", y_test.shape

X_train 크기:  (4524, 11)
y_train 크기:  (4524,)
X_val 크기:  (970, 11)
y_val 크기:  (970,)
X_test 크기:  (969, 11)
y_test 크기:  (969,)


In [8]:
# Modeling modules
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
import xgboost as xgb

## random forest, extra_trees, svm, mlp, adaboost, gradient boosting, xgbc

- 엑스트라 트리는 트리를 더욱 무작위하게 만들기 위해 최적의 임곗값을 찾는 대신에 후보 특성을 사용하여 무작위로 분할한 후 최상의 분할을 선택한다.
- 아다부스트는 반복마다 샘플의 가중치를 수정
- 그래디언트 부스팅은 아다부스트처럼 예측기를 순차적으로 추가, 이전 예측기의 잔여 오차에 새로운 예측기를 학습

In [12]:
random_forest_clf = RandomForestClassifier(n_estimators=10000, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=5000, random_state=42)
svm_clf = LinearSVC(max_iter=10000, random_state=42)
mlp_clf = MLPClassifier(max_iter=10000,random_state=42)
ada_clp=AdaBoostClassifier(n_estimators=10000,random_state=42)
gradient_boosting_clf=GradientBoostingClassifier(n_estimators=10000,random_state=42)
xgbc_clf=xgb.XGBClassifier(base_score=0.5, colsample_bylevel=1, n_jobs=1,
                       colsample_bytree=1, gamma=0.0001, max_delta_step=0, random_state=42, 
                       silent=True, subsample=1)

estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf, gradient_boosting_clf,xgbc_clf]
for estimator in estimators:
    print("Train prediction machine: ", estimator)
    estimator.fit(X_train, y_train)

[estimator.score(X_val, y_val) for estimator in estimators]

In [ ]:
random_forest_clf.fit(X_train, y_train)

In [ ]:
random_forest_clf.score(X_val, y_val)

In [10]:
extra_trees_clf.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=5000, n_jobs=None,
           oob_score=False, random_state=42, verbose=0, warm_start=False)

In [11]:
extra_trees_clf.score(X_val, y_val)

0.6865979381443299

In [13]:
###10000으로 하니까 셀이 자꾸 죽어서 나눴음
estimators2 = [svm_clf, mlp_clf]
for estimator in estimators2:
    print("Train prediction machine: ", estimator)
    estimator.fit(X_train, y_train)

('Train prediction machine: ', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=10000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0))


/home/ec2-user/anaconda3/envs/pytorch_p27/lib/python2.7/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


('Train prediction machine: ', MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=10000, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=42, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False))


In [14]:
[estimator.score(X_val, y_val) for estimator in estimators2]

[0.5030927835051546, 0.534020618556701]

In [15]:
###10000으로 하니까 셀이 자꾸 죽어서 나눴음
estimators3 = [gradient_boosting_clf,xgbc_clf]
for estimator in estimators3:
    print("Train prediction machine: ", estimator)
    estimator.fit(X_train, y_train)

('Train prediction machine: ', GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10000,
              n_iter_no_change=None, presort='auto', random_state=42,
              subsample=1.0, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False))
('Train prediction machine: ', XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0.0001, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subs

In [16]:
[estimator.score(X_val, y_val) for estimator in estimators3]

[0.6618556701030928, 0.5979381443298969]

## quality일 때  
#### 기본값-svc는 10000, 나머지는 아마 10?
[0.668041237113402,
0.6391752577319587,
0.5030927835051546,
0.534020618556701,
0.5989690721649484,
0.5979381443298969]

#### max_iter 1000으로
[0.6907216494845361,
0.6865979381443299,
0.3391752577319588,
0.534020618556701,
0.6608247422680412,
0.5979381443298969]

#### max_iter 10000으로
[0.6927835051546392, 0.6865979381443299(5000으로함), 0.5030927835051546, 0.534020618556701, 0.6618556701030928, 0.5979381443298969]

<br>
<br>

## 이진분류일 때
[0.7958762886597938,
0.8123711340206186,
0.6597938144329897,
0.7381443298969073,
0.7762886597938145,
0.7804123711340206]

#### max_iter 1000으로
[0.8216494845360824,
0.8257731958762886,
0.42061855670103093,
0.7381443298969073,
0.8134020618556701,
0.7804123711340206]

#### max_iter 10000으로
[0.822680412371134, 0.8288659793814434, 0.6597938144329897, 0.7381443298969073, 0.8103092783505155, 0.7804123711340206]